In [1]:
from pymongo import MongoClient
import pickle
from pprint import pprint
import config

from pandas import read_excel
import pandas as pd
import function as func
import datetime
from typing import List, Tuple
from db_connect import EngineConnect as DatabaseConnect
from sqlalchemy.sql import text

In [3]:
def get_process_id_performance(self, type):
    if type == 'keying':
            return 3
        elif type == 'qc':
            return 5
        elif type == 'approve_qc':
            return 6
     
def get_working_type_id_by_name(self, name: str):
    if name == 'In WorkShift':
        return 1
    else:
        return 2
    
    
    
    
    
data_model = DataModel()
dim_process = data_model.dim_process()
dim_work_type = data_model.dim_work_type()
data_model.update(dim_process, config.DWH_ANALYTIC_SCHEMA, config.DWH_DIM_PROCESS_TABLE)
data_model.update(dim_work_type, config.DWH_ANALYTIC_SCHEMA, config.DWH_DIM_WORK_TYPE_TABLE)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 4)

In [ ]:
class FactPerformanceModel:
    def __init__(
        self,
        *kwargs,
        ori_id: str,
        project_id: str,
        group_id: str = None,
        document_id: str = None,
        reworked: bool = False,
        work_type_id: int,
        process_key: int,
        number_of_record: int,
        user_name: str,
        ip: str = None,
        captured_date_timestamp: str,
        captured_date_key: int,
        captured_time_key: int,
        total_time_second: int
    ):
        self.ori_id = ori_id
        self.project_id = project_id
        self.group_id = group_id
        self.document_id = document_id
        self.reworked = reworked
        self.work_type_id = work_type_id        
        self.process_key = process_key
        self.number_of_record = number_of_record
        self.user_name = user_name
        self.ip = ip
        self.captured_date_timestamp = captured_date_timestamp
        self.captured_date_key = captured_date_key
        self.captured_time_key = captured_time_key
        self.total_time_second = total_time_second
        

class DimProcessModel:
    def __init__(
        self,
        *kwargs,
        id: str,
        module: str = None,
        step: str = None,
        sub_step: str = None,
        resource: str
     ):
        self.id = id
        self.module = type
        self.step = step
        self.sub_step = sub_step
        self.resource = resource

In [ ]:
pers_load = pickle.load(open('performance_eclaims.pickle', 'rb'))
data_performance = [item for item in pers_load]

In [ ]:
fact_performances = []

In [8]:
for performance in data_performance:
    performance_keys = performance.keys()
    ori_id = func.bson_object_to_string(performance['_id'])
    project_id = '5db144de27f919001f5f25e5'
    group_id = performance['group_id']
    document_id = performance['documentId']
    reworked = performance['has_rework']
    work_type_id = data_model.get_working_type_id_by_name(performance['work_type']) 
    user_name = performance['username']
    ip = None
    captured_date_timestamp = datetime.datetime.strptime(performance['captured_date'], '%d/%m/%Y')
    captured_date_key = func.time_to_date_key(captured_date_timestamp)
    captured_time_key = 0
    
    layout_name = performance['layout_name']
    module_type = performance['type'] 
    section_name = performance['section_name']
    if section_name != None:
        section_name = section_name[0]
    else:
        section_name = ''
        
    task_name = performance['task_name'] 
    process_key = data_model.get_process_id_performance(module_type)
    number_of_record = performance['records']
    total_time_second = performance['total_time']
    obj_ = FactPerformanceModel(
        ori_id = ori_id,  
        project_id = project_id,  
        group_id = group_id,  
        document_id = document_id,  
        reworked = reworked,  
        work_type_id = work_type_id,  
        process_key = process_key,  
        number_of_record = number_of_record,  
        user_name = user_name, 
        ip = ip , 
        captured_date_timestamp = captured_date_timestamp,  
        captured_date_key = captured_date_key,  
        captured_time_key = captured_time_key,  
        total_time_second = total_time_second 
    )
    fact_performances.append(obj_)
    
pprint(fact_performances[0].__dict__)
data_model.create([item.__dict__ for item in fact_performances], config.DWH_ANALYTIC_SCHEMA, config.DWH_FACT_PERFORMANCE_TABLE) 

{'captured_date_key': 20191203,
 'captured_date_timestamp': datetime.datetime(2019, 12, 3, 0, 0),
 'captured_time_key': 0,
 'document_id': '5de6246d62ca2e0011908f4e',
 'group_id': None,
 'ip': None,
 'number_of_record': 1,
 'ori_id': '5de6688d52439f0014f786d9',
 'process_id': 3,
 'project_id': '5db144de27f919001f5f25e5',
 'reworked': False,
 'total_time_second': 156289,
 'user_name': 'nhattta',
 'work_type_id': 1}
